In [4]:
# -*- coding: utf-8 -*-

%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

# <Step1. 크롤링> : 크롤링으로 웹 데이터 가져오기

In [5]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re

# 윈도우용 크롬 웹드라이버 실행 경로
# 크롬 드라이버를 다운 받고 그 위치를 적는다.
# 파일과 같은 위치로 옮겼기 때문에 아래의 주소를 적는다.
excutable_path = "./chromedriver.exe"

source_url = "https://namu.wiki/RecentChanges"

# 사이트의 html 구조에 기반하여 크롤링을 수행
driver = webdriver.Chrome(executable_path=excutable_path)
driver.get(source_url)

In [6]:
# 컨텐츠가 모두 로딩되기 전에 아래 코드들을 실행하면
# 제대로 된 결과가 나오지 않는다
# 따라서 페이지 로딩이 끝난 후 아래 코드들을 실행한다.
req = driver.page_source
soup = BeautifulSoup(req, "html.parser")
contents_table = soup.find(name="table")
table_body = contents_table.find(name='tbody')
table_rows = table_body.find_all(name='tr')

In [7]:
# a태그의 href 속성을 리스트로 추출하여, 크롤링 할 페이지 리스트를 생성
page_url_base = "https://namu.wiki"
page_urls = []
for index in range(0, len(table_rows)):
    first_td = table_rows[index].find_all("td")[0]
    td_url = first_td.find_all("a")
    if len(td_url) > 0:
        page_url = page_url_base + td_url[0].get("href")
        if "png" not in page_url:
            page_urls.append(page_url)

# 중복 url 제거
page_urls = list(set(page_urls))
for page in page_urls[:5]:
    print(page)
    
# 크롤링에 사용한 브라우저 종료
driver.close()

https://namu.wiki/w/%EB%9D%BC%EB%9D%BC(%EB%A9%94%EC%9D%B4%ED%94%8C%EC%8A%A4%ED%86%A0%EB%A6%AC)/%EC%97%AD%EC%82%AC
https://namu.wiki/w/%ED%9E%88%EA%B0%80%EC%8B%9C%EC%BD%94%EA%B0%80%EB%84%A4%EC%9D%B4%EC%97%AD
https://namu.wiki/w/%EC%A5%AC%EB%9D%BC%EA%B8%B0%20%EA%B3%B5%EC%9B%90(%EC%98%81%ED%99%94)
https://namu.wiki/w/%ED%8A%B8%EB%A6%AC%20%EC%98%A4%EB%B8%8C%20%EC%84%B8%EC%9D%B4%EB%B9%84%EC%96%B4/%EB%93%B1%EC%9E%A5%EC%9D%B8%EB%AC%BC/%EC%97%AC%EC%8B%A0%EC%9D%98%20%EC%84%B8%EB%A0%A5
https://namu.wiki/w/My%20Summer%20Car/%ED%8A%9C%EB%8B%9D%20%EC%9A%A9%ED%92%88


### [나무위키 최근변경 데이터 크롤링]

In [8]:
# 크롤링한 데이터를 데이터프레임으로 만들기위해 준비
colums = ['title', 'category', 'content_text']
df = pd.DataFrame(columns=colums)

# 각 페이지별 '제목', '카테고리', '본문' 정보를 데이터프레임으로 만듦
for page_url in page_urls:
    
    # 사이트의 html 구조에 기반하여 크롤링을 수행
    driver = webdriver.Chrome(executable_path=excutable_path)
    driver.get(page_url)
    req = driver.page_source
    soup = BeautifulSoup(req, "html.parser")
    contents_table = soup.find(name='article')
    title = contents_table.find_all('h1')[0]
    
    # 카테고리 정보가 없는 경우 확인
    if len(contents_table.find_all('h1')) > 0:
        category = contents_table.find_all('ul')[0]
    else:
        category = None
        
    content_paragraphs = contents_table.find_all(name='div', attrs={'class':'wiki-paragraph'})
    content_corpus_list = []
    
    # 페이지 내 제목 정보에서 개행 문자를 제거한 뒤 추출
    # 만약 없는 경우, 빈 문자열로 대체
    if title is not None:
        row_title = title.text.replace('\n', ' ')
    else:
        row_title = ""
        
    # 페이지 내 본문 정보에서 개행 문자를 제거한 뒤 추출
    # 만약 없는 경우, 빈 문자열로 대체
    if content_paragraphs is not None:
        for paragraphs in content_paragraphs:
            if paragraphs is not None:
                content_corpus_list.append(paragraphs.text.replace('\n', ' '))
            else:
                content_corpus_list.append("")
    else:
        content_corpus_list.appen("")
        
    # 페이지 내 카테고리 정보에서 "분류"라는 단어와 개행 문자를 제거한 뒤 추출
    # 만약 없는 경우, 빈 문자열로 대체
    if category is not None:
        row_category = category.text.replace("\n", " ")
    else:
        row_category = ""
        
    # 모든 정보를 하나의 데이터 프레임에 저장
    row = [row_title, row_category, "".join(content_corpus_list)]
    series = pd.Series(row, index=df.columns)
    df = df.append(series, ignore_index=True)
    
    # 크롤링에 사용한 브라우저 종료
    driver.close()

KeyboardInterrupt: 

In [9]:
df.head(5)

,title,category,content_text
0,라라(메이플스토리)/역사,메이플스토리/직업/역사/마법사,상위 문서: 라라(메이플스토리)메이플스토리 직업의 역사 [ 펼치기 · 접기 ]...
1,히가시코가네이역,일본의 철도역/ㅎ1964년 개업한 철도역츄오 본선츄오 쾌속선,東(ひがし)小(こ)金(が)井(ねい)駅(えき)Higashi-Koganei Statio...
2,쥬라기 공원(영화),쥬라기 공원 시리즈1993년 영화10억 달러 돌파 영화소설 원작 영화,스티븐 스필버그 감독 장편 연출 작품 (TV영화 제외) [ 펼치기 · 접기 ]상...


# <Step2. 추출> : 키워드 추출

### [텍스트 데이터 전처리]

In [10]:
# 텍스트 정제 함수 : 한글 이외의 문자는 전부 제거
def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')  # 한글의 정규 표현식
    result = hangul.sub('', text)
    return result

In [11]:
print(text_cleaning(df['content_text'][0]))

  상위 문서 라라메이플스토리메이플스토리 직업의 역사  펼치기  접기 전사 계열마법사 계열궁수 계열도적 계열해적 계열모험가히어로아크메이지불독보우마스터나이트로드바이퍼팔라딘아크메이지썬콜신궁섀도어캡틴다크나이트비숍패스파인더듀얼블레이드캐논슈터시그너스기사단소울마스터플레임위자드윈드브레이커나이트워커스트라이커미하일레지스탕스  데몬블래스터배틀메이지와일드헌터메카닉데몬슬레이어데몬어벤져제논영웅아란에반메르세데스팬텀은월루미너스노바카이저카인 카데나엔젤릭버스터레프아델일리움 아크아니마라라호영제로제로키네시스 키네시스전사 계열마법사 계열궁수 계열도적 계열해적 계열모험가히어로아크메이지불독보우마스터나이트로드바이퍼팔라딘아크메이지썬콜신궁섀도어캡틴다크나이트비숍패스파인더듀얼블레이드캐논슈터시그너스기사단소울마스터플레임위자드윈드브레이커나이트워커스트라이커미하일레지스탕스  데몬블래스터배틀메이지와일드헌터메카닉데몬슬레이어데몬어벤져제논영웅아란에반메르세데스팬텀은월루미너스노바카이저카인 카데나엔젤릭버스터레프아델일리움 아크아니마라라호영제로제로키네시스 키네시스 공개 이전 공개 직후테스트 서버 기준  이전에 추가된 직업인 아크 패파 아델이 출시 당시 상당히 강력한 성능을 보여준 것과 비교해서 애매한 성능을 보인다는 의견이 많다라라의 성능이나 사냥 효율은 평범한 수준이며 특정 캐릭터를 집중적으로 육성해오던 기존의 유저가 굳이 라라로 본캐를 갈아탈 동기부여도 부족하다는 평가가 많다 링크 스킬의 경우 상당한 비율의 일반 몬스터 데미지 증가 효과를 가지고 있어서 대부분의 라이트 유저와 메이플 유니온을 육성하는 유저들에겐 유용한 스킬이다전반적인 게임 성능의 평가는 긍정적이라고 보기 어렵지만 성능과는 별개로 캐릭터 스타일에 대한 평가는 좋다 스킬 이펙트가 매우 화려하고 멋있으며 귀여운 목소리를 가진 김가령 성우의 캐스팅도 호평을 받았고 캐릭터 디자인도 귀엽게 뽑혔다는 평가를 받는다이에 대해 메이플 유튜버인 이득충은 라라가 기존의 올드 유저층보다는 귀여운 디자인과 이펙트에 이끌려 직업을 선택하는 라이트 유저와 신규 유저를 겨냥한 직업일지도 모

In [12]:
# 각 피처마다 데이터 전처리를 적용
df['title'] = df['title'].apply(lambda x: text_cleaning(x))
df['category'] = df['category'].apply(lambda x: text_cleaning(x))
df['content_text'] = df['content_text'].apply(lambda x: text_cleaning(x))
df.head(5)

,title,category,content_text
0,라라메이플스토리역사,메이플스토리직업역사마법사,상위 문서 라라메이플스토리메이플스토리 직업의 역사 펼치기 접기 전사 계열마법...
1,히가시코가네이역,일본의 철도역ㅎ년 개업한 철도역츄오 본선츄오 쾌속선,히가시니혼 히가시코가네이역도쿄 방면무사시사카이 츄오 쾌속선 타카오 방면무사시코...
2,쥬라기 공원영화,쥬라기 공원 시리즈년 영화억 달러 돌파 영화소설 원작 영화,스티븐 스필버그 감독 장편 연출 작품 영화 제외 펼치기 접기 상세정보 링크 ...


### [말뭉치 만들기]

In [13]:
# 각 피처마다 말뭉치를 생성
title_corpus = "".join(df['title'].tolist())
catagory_corpus = "".join(df['category'].tolist())
content_corpus = "".join(df['content_text'].tolist())
print(title_corpus)

라라메이플스토리역사 히가시코가네이역 쥬라기 공원영화 


### [konply를 이용한 키워드 추출]

In [ ]:
# konlpy 사용하기가 되게 힘들다
# java path 설정
# Jpype1 version 0.7.0 사용
# konlpy version 0.5.1 사용
# https://m.blog.naver.com/kiddwannabe/221795961189

In [14]:
from konlpy.tag import Okt
from collections import Counter

# konlpy의 형태소 분석기로 명사 단위의 키워드를 추출
nouns_tagger = Okt()
nouns = nouns_tagger.nouns(content_corpus)
count = Counter(nouns)

In [15]:
count

Counter({'상위': 1,
         '문서': 2,
         '라라': 7,
         '메이플스토리': 2,
         '직업': 4,
         '역사': 2,
         '접기': 6,
         '전사': 2,
         '계열': 10,
         '마법사': 2,
         '궁수': 2,
         '도적': 2,
         '해적': 2,
         '모험가': 2,
         '히어로': 2,
         '아크메이지': 4,
         '불독': 2,
         '보우': 2,
         '마스터': 4,
         '나이트': 5,
         '로드': 2,
         '바이퍼': 2,
         '팔라딘': 2,
         '콜': 2,
         '신궁': 2,
         '섀도': 2,
         '캡틴': 2,
         '다크나이트': 2,
         '비숍': 2,
         '패스파인더': 2,
         '듀얼': 2,
         '블레이드': 2,
         '캐논': 2,
         '슈터': 2,
         '시그너스': 2,
         '기사단': 2,
         '소울': 2,
         '플레임': 2,
         '위자드': 2,
         '윈드브레이커': 2,
         '워커': 2,
         '스트라이커': 2,
         '미하일': 2,
         '레지스탕스': 2,
         '데몬': 6,
         '블': 2,
         '래스터': 2,
         '배틀': 4,
         '메이지': 2,
         '와일드': 2,
         '헌터': 2,
         '메카': 2,
         '닉': 2,
       